In [102]:
import pandas as pd
import ast
import nltk
import ssl
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import jaccard_score

In [3]:
# Download necessary nltk packages for language processing
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [92]:
movies_df = pd.read_csv("final_movie.csv")

In [93]:
movies_df


movies_df['cast'].fillna("", inplace=True) # replace missing values with empty string
movies_df['keywords'].fillna("", inplace=True) # replace missing values with empty string




In [94]:
movies_df

,id,keywords,cast,Director,genres,imdb_id,original_language,revenue,release_date,spoken_languages,title,vote_average,vote_count,overview,poster_path,popularity
0,862,jealousy toy boy friendship friends rivalry bo...,TomHanks TimAllen DonRickles JimVarney Wallace...,['John Lasseter'],"['Animation', 'Comedy', 'Family']",tt0114709,en,373554033.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,21.946943
1,8844,boardgame disappearance basedonchildren'sbook ...,RobinWilliams JonathanHyde KirstenDunst Bradle...,['Joe Johnston'],"['Adventure', 'Fantasy', 'Family']",tt0113497,en,262797249.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,17.015539
2,15602,fishing bestfriend duringcreditsstinger oldmen,WalterMatthau JackLemmon Ann-Margret SophiaLor...,['Howard Deutch'],"['Romance', 'Comedy']",tt0113228,en,0.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,11.712900
3,31357,basedonnovel interracialrelationship singlemot...,WhitneyHouston AngelaBassett LorettaDevine Lel...,['Forest Whitaker'],"['Comedy', 'Drama', 'Romance']",tt0114885,en,81452156.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom...",/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,3.859495
4,11862,baby midlifecrisis confidence aging daughter m...,SteveMartin DianeKeaton MartinShort KimberlyWi...,['Charles Shyer'],['Comedy'],tt0113041,en,76578911.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...,/e64sOI48hQXyru7naBFyssKFxVd.jpg,8.387519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44099,222848,,LisaBoyle KenaLand ZanetaPolard DonYanan Debra...,['Aaron Osborne'],['Science Fiction'],tt0112613,en,0.0,1995,"[{'iso_639_1': 'en', 'name': 'English'}]",Caged Heat 3000,3.5,1.0,It's the year 3000 AD. The world's most danger...,/4lF9LH0b0Z1X94xGK9IOzqEW6k1.jpg,0.661558
44100,30840,,PatrickBergin UmaThurman DavidMorrissey Jürgen...,['John Irvin'],"['Drama', 'Action', 'Romance']",tt0102797,en,0.0,1991,"[{'iso_639_1': 'en', 'name': 'English'}]",Robin Hood,5.7,26.0,"Yet another version of the classic epic, with ...",/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,5.683753
44101,439050,tragiclove,LeilaHatami KouroshTahami ElhamKorda,['Hamid Nematollah'],"['Drama', 'Family']",tt6209470,fa,0.0,NaT,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Subdue,4.0,1.0,Rising and falling between a man and woman.,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,0.072051
44102,111109,artist play pinoy,AngelAquino PerryDizon HazelOrencio JoelTorre ...,['Lav Diaz'],['Drama'],tt2028550,tl,0.0,2011,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,9.0,3.0,An artist struggles to finish his work while a...,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,0.178241


In [95]:
# Processing text data
def preprocess_overview(text):

    if not isinstance(text, str):
        return []
    
    # tokenize the text
    tokens = word_tokenize(text)

     # Convert to lowercase and remove non-alphanumeric characters
    words = [word.lower() for word in tokens if word.isalnum()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return lemmatized_words

In [99]:
def extract_keywords(row):
    keywords = str(row['keywords'])
    
    
    directors = ' '.join(ast.literal_eval(row['Director']))
    
    overview = row['overview']
    overview_keywords = ' '.join(preprocess_overview(overview))

    genres = ' '.join(ast.literal_eval(row['genres'])).lower()

    year = row['release_date']

    cast = row['cast']


    
    combined_keywords = ' '.join([genres, keywords, year, overview_keywords, directors, cast])
    
    return combined_keywords


In [110]:

df_movies = pd.DataFrame()


df_movies['combined_keywords'] = movies_df.apply(extract_keywords, axis=1)
df_movies['title'] = movies_df['title']

In [111]:
df_movies

,combined_keywords,title
0,animation comedy family jealousy toy boy frien...,Toy Story
1,adventure fantasy family boardgame disappearan...,Jumanji
2,romance comedy fishing bestfriend duringcredit...,Grumpier Old Men
3,comedy drama romance basedonnovel interracialr...,Waiting to Exhale
4,comedy baby midlifecrisis confidence aging dau...,Father of the Bride Part II
...,...,...
44099,science fiction 1995 year 3000 ad world dange...,Caged Heat 3000
44100,drama action romance 1991 yet another version...,Robin Hood
44101,drama family tragiclove NaT rising falling man...,Subdue
44102,drama artist play pinoy 2011 artist struggle f...,Century of Birthing


In [107]:
stop = list(stopwords.words('english'))

tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', stop_words=list(set(stop)))
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies['combined_keywords'])

cosine_sim = cosine_similarity(tfidf_matrix)

In [112]:
def recommend_movies(title, num_recommendations=10):
    # Get the index of the movie with the given title
    movie_index = df_movies[df_movies['title'] == title].index[0]

    # Get the list of cosine similarity scores for the given movie
    similarity_scores = list(enumerate(cosine_sim[movie_index]))

    # Sort the scores in descending order
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the most similar movies (excluding the input movie itself)
    similar_movie_indices = [score[0] for score in similarity_scores_sorted[1:num_recommendations + 1]]

    # Return the titles of the most similar movies
    return df_movies['title'].iloc[similar_movie_indices]

In [114]:
recommended_movies = recommend_movies("Toy Story", 10)
print(recommended_movies)

2951                                      Toy Story 2
15255                                     Toy Story 3
23995                                       Small Fry
21551                            Toy Story of Terror!
25191                      Toy Story That Time Forgot
36157                                  Toy Reanimator
23993                               Hawaiian Vacation
23351                     Andy Hardy's Blonde Trouble
25095     Silent Night, Deadly Night 5: The Toy Maker
41578    Andy Peters: Exclamation Mark Question Point
Name: title, dtype: object
